# Análisis de Mortalidad Fetal en Guatemala (2012-2022) mediante Clustering K-Means

Daniel Chet - 231177, Dulce Ambrosio - 231143

---

### Situación Problemática

Guatemala registra una de las tasas de mortalidad fetal más altas de Latinoamérica. El presente proyecto busca analizar los datos oficiales de **Defunciones Fetales y Nacimientos (2012-2022)** publicados por el Instituto Nacional de Estadística (INE) para identificar patrones y perfiles de riesgo mediante técnicas de análisis exploratorio y minería de datos (K-Means). El objetivo es descubrir si existen segmentos diferenciados de la población materna que presenten mayor vulnerabilidad a la muerte fetal, considerando variables como edad, etnia, escolaridad, asistencia recibida y semanas de gestación.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Configuración de estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_columns', None)

: 

## 2. Preprocesamiento de Datos

El preprocesamiento enfrenta varios retos importantes:
- Los archivos `.sav` (SPSS) de diferentes años tienen **nombres de columnas inconsistentes** (mayúsculas, minúsculas, abreviaciones distintas).
- Algunas variables contienen valores como **"Ignorado"** en lugar de datos numéricos.
- Existen **datos atípicos** en edad materna (valores como 99 o 999 que son códigos de "no reportado").
- El peso al nacer viene separado en libras y onzas, y debe consolidarse.
- Las semanas de gestación a veces vienen como texto y requieren conversión numérica.

### 2.0 Exploración de columnas originales

Antes de unificar los datos, es necesario revisar qué columnas tiene cada archivo y detectar cambios en los nombres entre años. Esto nos permite construir el diccionario de mapeo que usaremos después para estandarizar.

In [ ]:
# --- Exploración de columnas: Defunciones Fetales ---
import pandas as pd
import os

df_temp = pd.read_spss("DefuncionesFetales/DF2012.sav")
pd.set_option('display.max_columns', None)
print("Columnas de DF2012:")
print(df_temp.columns.tolist())
df_temp.info()

In [ ]:
# --- Comparación de columnas entre años: Defunciones Fetales ---
ROJO = "\033[91m"
VERDE = "\033[92m"
AMARILLO = "\033[93m"
AZUL = "\033[94m"
RESET = "\033[0m"

carpeta = "DefuncionesFetales" 
archivos = [f for f in os.listdir(carpeta) if f.endswith('.sav')]
archivos.sort()

df_prev = pd.read_spss(os.path.join(carpeta, archivos[0]))
cols_prev = set(df_prev.columns)

print(f"{AZUL}Base: {archivos[0]} tiene {len(cols_prev)} columnas.{RESET}")

for archivo in archivos[1:]:
    print(f"\n{AZUL}--- Comparando {archivo} vs año anterior ---{RESET}")
    
    try:
        df_actual = pd.read_spss(os.path.join(carpeta, archivo))
        cols_actual = set(df_actual.columns)
        
        nuevas = cols_actual - cols_prev
        if nuevas:
            print(f"{AMARILLO}AGREGADAS: {nuevas}{RESET}")
            
        perdidas = cols_prev - cols_actual
        if perdidas:
            print(f"{ROJO}ELIMINADAS (o cambiaron nombre): {perdidas}{RESET}")
            
        if not nuevas and not perdidas:
            print(f"{VERDE}Estructura idéntica.{RESET}")
            
        cols_prev = cols_actual
        
    except Exception as e:
        print(f"{ROJO}Error leyendo {archivo}: {e}{RESET}")

In [ ]:
# --- Exploración de columnas: Nacimientos ---
df_temp = pd.read_spss("Nacimientos/N2012.sav")
pd.set_option('display.max_columns', None)
print("Columnas de N2012:")
print(df_temp.columns.tolist())
df_temp.info()

In [ ]:
# --- Comparación de columnas entre años: Nacimientos ---
carpeta = "Nacimientos" 
archivos = [f for f in os.listdir(carpeta) if f.endswith('.sav')]
archivos.sort()

df_prev = pd.read_spss(os.path.join(carpeta, archivos[0]))
cols_prev = set(df_prev.columns)

print(f"{AZUL}Base: {archivos[0]} tiene {len(cols_prev)} columnas.{RESET}")

for archivo in archivos[1:]:
    print(f"\n{AZUL}--- Comparando {archivo} vs año anterior ---{RESET}")
    
    try:
        df_actual = pd.read_spss(os.path.join(carpeta, archivo))
        cols_actual = set(df_actual.columns)
        
        nuevas = cols_actual - cols_prev
        if nuevas:
            print(f"{AMARILLO}AGREGADAS: {nuevas}{RESET}")
            
        perdidas = cols_prev - cols_actual
        if perdidas:
            print(f"{ROJO}ELIMINADAS (o cambiaron nombre): {perdidas}{RESET}")
            
        if not nuevas and not perdidas:
            print(f"{VERDE}Estructura idéntica.{RESET}")
            
        cols_prev = cols_actual
        
    except Exception as e:
        print(f"{ROJO}Error leyendo {archivo}: {e}{RESET}")

### 2.1 Lectura de archivos de Nacimientos (2012-2022)

In [ ]:
# --- CONFIGURACIÓN ---
carpeta_nacimientos = "./Nacimientos/"

# DICCIONARIO DE COLUMNAS A CONSERVAR
cols_nacimientos_map = {
    # --- Geografía (Residencia es mejor que Ocurrencia para análisis social) ---
    'Departamento': ['Deprem', 'DEPREM'],
    'Municipio': ['Muprem', 'MUPREM'],
    
    # --- Tiempo ---
    'Anio': ['Añoocu', 'AÑOOCU', 'Añoreg', 'AÑOREG', 'Anoreg'],
    'Mes': ['Mesocu', 'MESOCU', 'Mesreg', 'MESREG'],
    
    # --- Datos Madre ---
    'Edad_Madre': ['Edadm', 'EDADM'],
    'Estado_Civil': ['Escivm', 'ESCIVM'],
    'Escolaridad': ['Escolam', 'ESCOLAM'],
    'Total_Hijos': ['Tohite', 'TOHITE'],
    'Pueblo_Etnia': ['PuebloPM', 'PUEBLOPM', 'Pueblopm', 'grupetma', 'GRUPETMA'],
    
    # --- Datos Bebé ---
    'Sexo': ['Sexo', 'SEXO'],
    'Tipo_Parto': ['Tipar', 'TIPAR'],
    'Sitio_Ocurrencia': ['Sitioocu', 'SITIOOCU'],
    'Asistencia_Recibida': ['Asisrec', 'ASISREC'],
    
    # --- Peso (temporales para cálculo) ---
    'Libras_Temp': ['Libras', 'LIBRAS'],
    'Onzas_Temp': ['Onzas', 'ONZAS']
}

def procesar_nacimientos(carpeta, diccionario_map):
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.sav')]
    archivos.sort()
    archivos = [f for f in archivos if any(str(y) in f for y in range(2012, 2023))]
    
    lista_dfs = []
    print(f"Procesando {len(archivos)} archivos de Nacimientos (2012-2022)...")
    
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        try:
            df_raw = pd.read_spss(ruta, convert_categoricals=True)
            df_limpio = pd.DataFrame()
            
            for nombre_final, variantes in diccionario_map.items():
                col_encontrada = None
                for col_real in df_raw.columns:
                    if col_real.upper() in [v.upper() for v in variantes]:
                        col_encontrada = col_real
                        break
                if col_encontrada:
                    df_limpio[nombre_final] = df_raw[col_encontrada]
                else:
                    df_limpio[nombre_final] = np.nan
            
            lista_dfs.append(df_limpio)
            print(f"  -> {archivo}: OK")
        except Exception as e:
            print(f"  ERROR en {archivo}: {e}")

    if not lista_dfs:
        return None

    df_final = pd.concat(lista_dfs, ignore_index=True)
    
    print("\nCalculando pesos y limpiando...")
    df_final['Libras_Temp'] = pd.to_numeric(df_final['Libras_Temp'], errors='coerce').fillna(0)
    df_final['Onzas_Temp'] = pd.to_numeric(df_final['Onzas_Temp'], errors='coerce').fillna(0)
    df_final['Peso_Libras'] = df_final['Libras_Temp'] + (df_final['Onzas_Temp'] / 16)
    df_final = df_final.drop(columns=['Libras_Temp', 'Onzas_Temp'])
    df_final = df_final[df_final['Peso_Libras'] > 0]

    return df_final

# --- EJECUTAR ---
df_nacimientos = procesar_nacimientos(carpeta_nacimientos, cols_nacimientos_map)

print("\n--- RESUMEN FINAL ---")
print(df_nacimientos.info())
print("\nPrimeras 5 filas:")
print(df_nacimientos.head())

### 2.2 Lectura de archivos de Defunciones Fetales (2012-2022)

In [ ]:
import pandas as pd
import os
import numpy as np

# --- CONFIGURACIÓN ---
# Ajusta la ruta a donde tengas tus archivos de DEFUNCIONES
carpeta_defunciones = "./DefuncionesFetales/" 

# DICCIONARIO DE COLUMNAS A CONSERVAR (Defunciones Fetales)
cols_defunciones_map = {
    # --- Geografía (Usamos Residencia para cruzar con Nacimientos) ---
    'Departamento': ['DEPREM', 'Deprem'],
    'Municipio': ['MUPREM', 'Muprem'],
    
    # --- Tiempo ---
    # Prioridad: Ocurrencia. Si no existe, Registro.
    'Anio': ['AÑOOCU', 'Añoocu', 'AÑOREG', 'Añoreg', 'ANOREG'],
    'Mes': ['MESOCU', 'Mesocu', 'MESREG', 'Mesreg'],
    
    # --- Datos Madre ---
    'Edad_Madre': ['EDADM', 'Edadm'],
    'Estado_Civil': ['ESCIVM', 'Escivm'],
    'Escolaridad': ['ESCOLAM', 'Escolam'],
    'Total_Hijos': ['TOHITE', 'Tohite'],
    # El cambio difícil: GRETNM -> PUEBLOPM -> NACIONM (a veces)
    'Pueblo_Etnia': ['PUEBLOPM', 'PuebloPM', 'Pueblopm', 'GRETNM', 'Gretnm'],
    
    # --- Datos Feto/Evento ---
    'Sexo': ['SEXO', 'Sexo'],
    'Tipo_Parto': ['TIPAR', 'Tipar'],
    'Sitio_Ocurrencia': ['SITIOOCU', 'Sitioocu'],
    'Asistencia_Recibida': ['ASISREC', 'Asisrec'],
    'Causa_Defuncion': ['CAUDEF', 'Caudef'], # Muy importante en defunciones
    
    # --- Variable Numérica Crítica ---
    'Semanas_Gestacion': ['SEMGES', 'Semges']
}

def procesar_defunciones(carpeta, diccionario_map):
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.sav')]
    archivos.sort()
    
    # Filtro 2012-2022
    archivos = [f for f in archivos if any(str(y) in f for y in range(2012, 2023))]
    
    lista_dfs = []
    print(f"Procesando {len(archivos)} archivos de Defunciones (2012-2022)...")
    
    for archivo in archivos:
        ruta = os.path.join(carpeta, archivo)
        try:
            # Leer archivo con etiquetas
            df_raw = pd.read_spss(ruta, convert_categoricals=True)
            
            df_limpio = pd.DataFrame()
            
            for nombre_final, variantes in diccionario_map.items():
                col_encontrada = None
                for col_real in df_raw.columns:
                    if col_real.upper() in [v.upper() for v in variantes]:
                        col_encontrada = col_real
                        break
                
                if col_encontrada:
                    df_limpio[nombre_final] = df_raw[col_encontrada]
                else:
                    df_limpio[nombre_final] = np.nan
            
            lista_dfs.append(df_limpio)
            print(f"  -> {archivo}: OK")
            
        except Exception as e:
            print(f"  ERROR en {archivo}: {e}")

    if not lista_dfs:
        return None

    # 1. Unir todo
    df_final = pd.concat(lista_dfs, ignore_index=True)
    
    # 2. LIMPIEZA ESPECÍFICA DE DEFUNCIONES
    print("\nLimpiando datos numéricos...")
    
    # Semanas de Gestación: Debe ser numérico.
    # A veces viene como "Ignorado" o texto. 'coerce' lo vuelve NaN.
    df_final['Semanas_Gestacion'] = pd.to_numeric(df_final['Semanas_Gestacion'], errors='coerce')
    
    # Edad Madre: Asegurar numérico
    df_final['Edad_Madre'] = pd.to_numeric(df_final['Edad_Madre'], errors='coerce')
    
    # Eliminar registros sin semanas de gestación válidas (opcional, pero recomendado para K-Means)
    df_final = df_final.dropna(subset=['Semanas_Gestacion'])

    return df_final

# --- EJECUTAR ---
df_defunciones = procesar_defunciones(carpeta_defunciones, cols_defunciones_map)

# Verificación
if df_defunciones is not None:
    print("\n--- RESUMEN FINAL DEFUNCIONES ---")
    print(df_defunciones.info())
    print("\nPrimeras 5 filas:")
    print(df_defunciones.head())

### 2.3 Estandarización de nombres y limpieza de variables numéricas

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo (para que se vean profesionales como en el notebook de clase)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
import pandas as pd
import numpy as np

# 1. Limpiar Edad en Nacimientos (Convertir "Ignorado" a NaN)
# errors='coerce' fuerza a que cualquier texto se vuelva un número vacío (NaN)
df_nacimientos['Edad_Madre'] = pd.to_numeric(df_nacimientos['Edad_Madre'], errors='coerce')

# 2. Eliminar las filas que quedaron vacías (NaN) en esa columna
df_nacimientos = df_nacimientos.dropna(subset=['Edad_Madre'])

# 3. (Opcional pero recomendado) Hacer lo mismo con Defunciones por seguridad
df_defunciones['Edad_Madre'] = pd.to_numeric(df_defunciones['Edad_Madre'], errors='coerce')
df_defunciones = df_defunciones.dropna(subset=['Edad_Madre'])

print("Limpieza de Edad completada.")

In [ ]:
print("--- Limpieza de Valores Atípicos en Peso ---")

# Ver cuántos datos raros hay antes de borrar
pesos_raros = df_nacimientos[df_nacimientos['Peso_Libras'] > 15].shape[0]
print(f"Se encontraron {pesos_raros} registros con peso mayor a 15 libras. Serán eliminados.")

# Filtrar para mantener solo pesos biológicamente posibles (ej. entre 1 y 15 libras)
df_nacimientos = df_nacimientos[(df_nacimientos['Peso_Libras'] >= 1) & (df_nacimientos['Peso_Libras'] <= 15)]

print("Limpieza de peso completada. Los QQ-Plots ahora se verán normales.")

In [ ]:
print("--- Estandarización Profunda de Variables Categóricas ---")

# 1. Diccionario extendido para Asistencia Recibida
mapeo_asistencia_ext = {
    'Médico': 'Médica',
    'Medico': 'Médica',
    'Empírico': 'Empírica',
    'Empirico': 'Empírica',
    'Paramédico': 'Paramédica',
    'Paramedico': 'Paramédica',
    'Ninguno': 'Ninguna',
    'Ignorada': 'Ignorado'
}

# 2. Diccionario extendido para Etnia
# Asumimos 'Indigena' como 'Maya' (el grupo mayoritario) y 'No indigena' como 'Mestizo / Ladino'
mapeo_etnia_ext = {
    'Xinca': 'Xinka',
    'Indigena': 'Maya',
    'No indigena': 'Mestizo / Ladino'
}

# 3. Aplicar los mapeos a Nacimientos
df_nacimientos['Asistencia_Recibida'] = df_nacimientos['Asistencia_Recibida'].replace(mapeo_asistencia_ext)
df_nacimientos['Pueblo_Etnia'] = df_nacimientos['Pueblo_Etnia'].replace(mapeo_etnia_ext)

# 4. Aplicar los mapeos a Defunciones (para mantener coherencia entre ambas tablas)
df_defunciones['Asistencia_Recibida'] = df_defunciones['Asistencia_Recibida'].replace(mapeo_asistencia_ext)
df_defunciones['Pueblo_Etnia'] = df_defunciones['Pueblo_Etnia'].replace(mapeo_etnia_ext)

print("¡Limpieza profunda completada!")

In [ ]:
import pandas as pd
import numpy as np

# Función para limpiar columnas numéricas a la fuerza
def limpiar_numericos(df, columnas):
    for col in columnas:
        # Convertir a numérico, errores se vuelven NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# 1. Definir las columnas numéricas que vas a usar
cols_num_nac = ['Edad_Madre', 'Peso_Libras', 'Total_Hijos']
cols_num_def = ['Edad_Madre', 'Semanas_Gestacion', 'Total_Hijos']

# 2. Aplicar limpieza
df_nacimientos = limpiar_numericos(df_nacimientos, cols_num_nac)
df_defunciones = limpiar_numericos(df_defunciones, cols_num_def)

# 3. Eliminar filas que quedaron vacías por tener datos inválidos
df_nacimientos = df_nacimientos.dropna(subset=cols_num_nac)
df_defunciones = df_defunciones.dropna(subset=cols_num_def)

print("¡Limpieza de variables numéricas completada!")

## 3. Análisis Exploratorio (EDA)

En esta sección se describen las variables numéricas y categóricas principales de ambos conjuntos de datos, evaluando su distribución, normalidad y posibles datos atípicos.

### 3.1 Descripción del conjunto de datos

In [ ]:
# Función para describir un dataset
def describir_dataset(df, nombre):
    print(f"--- DESCRIPCIÓN DE {nombre.upper()} ---")
    print(f"Total de Observaciones (Filas): {df.shape[0]}")
    print(f"Total de Variables (Columnas): {df.shape[1]}")
    print("\nTipos de Variables:")
    print(df.dtypes)
    print("\n")

# Ejecutar para ambos
describir_dataset(df_nacimientos, "Nacimientos")
describir_dataset(df_defunciones, "Defunciones Fetales")

### 3.2 Resumen estadístico y prueba de normalidad

In [ ]:
# 1. Resumen Estadístico (Media, Mediana, Desviación)
print("--- ESTADÍSTICAS DESCRIPTIVAS: NACIMIENTOS ---")
cols_num_nac = ['Edad_Madre', 'Peso_Libras', 'Total_Hijos']
print(df_nacimientos[cols_num_nac].describe().round(2))
print("\nAsimetría (Skewness):")
print(df_nacimientos[cols_num_nac].skew().round(2))

print("\n\n--- ESTADÍSTICAS DESCRIPTIVAS: DEFUNCIONES ---")
cols_num_def = ['Edad_Madre', 'Semanas_Gestacion', 'Total_Hijos']
print(df_defunciones[cols_num_def].describe().round(2))
print("\nAsimetría (Skewness):")
print(df_defunciones[cols_num_def].skew().round(2))

# 2. Prueba Visual de Normalidad (QQ-Plots)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

stats.probplot(df_nacimientos['Edad_Madre'], dist="norm", plot=axes[0])
axes[0].set_title("QQ-Plot: Edad Madre (Nacimientos)")

stats.probplot(df_nacimientos['Peso_Libras'], dist="norm", plot=axes[1])
axes[1].set_title("QQ-Plot: Peso al Nacer")

stats.probplot(df_defunciones['Semanas_Gestacion'], dist="norm", plot=axes[2])
axes[2].set_title("QQ-Plot: Semanas Gestación")

plt.tight_layout()
plt.show()